# Install Dependencies

In [9]:
!pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface --quiet
!pip install llama-index-llms-huggingface-api --quiet
! pip install transformers  evaluate accelerate --quiet
!pip install -U bitsandbytes --quiet
!pip install -q hazm
!pip install -q clean-text[gpl]
!pip install docx2txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

# Load Documents

In [14]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

document_1 = SimpleDirectoryReader(input_files=["/content/input_1.docx"]).load_data()
document_2 = SimpleDirectoryReader(input_files=["/content/input_2.docx"]).load_data()

In [27]:
def extract_text(documents):
    text = ""
    for doc in documents:
        text += doc.text + " "  # Assuming each document object has a 'text' attribute
    return text.strip()

def chunk_text(text, chunk_size=100):
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

text_1 = extract_text(document_1)
text_2 = extract_text(document_2)



# Preprocess Texts

In [34]:
import hazm
from cleantext import clean
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()

    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)

    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)

    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

In [35]:
text_1 = cleaning(text_1)
text_2 = cleaning(text_2)

In [36]:
chunks_2 = chunk_text(text_2, chunk_size=1070)
chunks_2.append(text_1)

# Initialize Retrieval Model

In [83]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [84]:
embeddings = model.encode(chunks_2, convert_to_tensor=True)

In [85]:
from google.colab import drive
import torch

# Mount Google Drive
drive.mount('/content/drive')

save_path = '/content/drive/My Drive/chunk_embeddings.pt'

torch.save(embeddings, save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
embeddings = torch.load(save_path)

# Define the retriever function

In [87]:
from sentence_transformers import util
import torch


def find_best_matching_document(query, model, embeddings, articles):
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    best_match_index = int(similarity_scores.argmax())
    return articles[best_match_index]

query = "چگونه میتوانم از صورت جلسات ماه ۵ پرینت بگیرم؟"
best_matching_document = find_best_matching_document(query, model, embeddings, chunks_2)
print("Best Matching Document:\n", best_matching_document)

Best Matching Document:
 اشغال کلیه طبقات، تحت هر عنوان و به هر شکل، بیش از میزان تعیینشده در جداول چهارگانه استفاده از اراضی پهنههای اصلی و مستثنی شده در این بند، ممنوع است. (اصلاح طبق بند (۶) مصوبه مورخ ۰۵ / ۰۸ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۸ - ۳: در کلیه زیرپهنههای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنههای r ۲، s ۳، m ۲ و g، با کاهش سطح اشغال طبقات و با رعایت سقف تراکم مجاز، به منظور تحققپذیری راهبردهای طرح جامع شهر تهران، افزایش طبقه بر اساس دستورالعملی که به تصویب کمیسیون ماده پنج خواهد رسید، مجاز خواهد بود. (اصلاح طبق بند (۷) مصوبه مورخ ۰۵ / ۰۸ / ۱۳۹۳ شورایعالی شهرسازی و معماری ایران) تبصره (۲): حذف تبصره به دلیل مغایرت با متن بند فوق طبق بند (۱) مصوبه مورخ ۲۸ / ۰۷ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران ۸ - ۴: در کلیه پهنهها، تخصیص حداقل ۵۰ درصد از سطح باز قطعه (پس از کسر مساحت رامپ) به فضای سبز و مشجر، الزامی است. در پهنههای s ۳۱، ضوابط مندرج دربند ۱۱ - ۳ ملاک عمل است. ۸ - ۵: سطح اشغال زیرزمین قطعات واقع در پهنههای فعالیت و مختلط، (صرفا به منظور استقر

# Define Generative Model

In [88]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, pipeline
from datasets import load_dataset


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPT-2 Small
model_name = "gpt2"
tokenizer1 = AutoTokenizer.from_pretrained(model_name)
tokenizer1.pad_token = tokenizer1.eos_token
model1 = GPT2LMHeadModel.from_pretrained(model_name)

In [92]:
def QA_gpt2(query, article):
    prompt = f"بهترین پاسخ برای سوال روبه‌رو باتوجه به متن داده شده چیست؟ \n سوال: {query}\n\nمتن: {article}"
    inputs = tokenizer1.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
    ids = model1.generate(inputs, max_length=1024, num_return_sequences=1)
    output = tokenizer1.decode(ids[0], skip_special_tokens=True)
    return output

# Test with an example query
query = "چگونه میتوانم از صورت جلسات ماه ۵ پرینت بگیرم؟"
best_matching_document = find_best_matching_document(query, model, embeddings, chunks_2)
summary_gpt2 = QA_gpt2(query, best_matching_document)

print(f"{summary_gpt2}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summary: بهترین پاسخ برای سوال روبه‌رو باتوجه به متن داده شده چیست؟ 
 سوال: چگونه میتوانم از صورت جلسات ماه ۵ پرینت بگیرم؟

متن: اشغال کلیه طبقات، تحت هر عنوان و به هر شکل، بیش از میزان تعیینشده در جداول چهارگانه استفاده از اراضی پهنههای اصلی و مستثنی شده در این بند، ممنوع است. (اصلاح طبق بند (۶) مصوبه مورخ ۰۵ / ۰۸ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۸ - ۳: در کلیه زیرپهنههای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنههای r ۲، s ۳، m ۲ و g، با کاه کلیه به من در جداول چهارگانه استفاده از اراضی پهنهای اصلی و مستثنی شده در این بند، ممنوع استفاده از اراضی، به استثنای محدودههای تاریخی و معماری ایران) ۸ - ۳: در کلیه زیرپهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای اصلی و مستثنی شده در این بند، ممنوع استفاده از اراضی، به استثنای محدودههای تاریخی و معماری ایران) ۸ - ۳: در کلیه زیرپهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و معماری ایران) ۸ - ۳: در کلیه زیرپهنهای استفا�


# Evaluation

In [93]:
pip install pandas

In [100]:
import random
import pandas as pd

# Replace 'questions.xlsx' with the path to your Excel file
file_path = 'questions.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# Assuming the questions are in a column named 'Questions'
questions_list = df['question'].tolist()
filtered_questions = [q for q in questions_list if len(q.split()) < 20]
random_questions = random.sample(filtered_questions, 10)

# Display the list of questions
print(random_questions)

['شهربان و نقش آن در کنترل و نظارت برساخت و سازهای شهر چیست؟', 'هزینه مشاعات ساختمان بر عهده کیست؟ ', ' تفکیک و افراز املاک چگونه است؟', 'تعریف بافت فرسوده و ناپایدار چیست ؟', 'انواع پروانه های ساختمانی را نام ببرید؟', 'احداث کنسول برروی چه گذرهایی و به چه میزان مجاز میباشد؟', 'مدارک لازم جهت صدور فیش های درآمد و نوسازی', 'آیا احداث استخر و سونا و جکوزی و مشتقات آن در زیرزمین مجاز است؟', 'دسترسی و نورگیری از فضای سبز عمومی چگونه است؟', 'مدارک مورد نیاز برای پیش نویس پایان کار ساختمان  کدام است؟']


In [102]:
for query in random_questions:
  best_matching_document = find_best_matching_document(query, model, embeddings, chunks_2)
  output_gpt2 = QA_gpt2(query, best_matching_document)
  print(f"\n{output_gpt2}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



بهترین پاسخ برای سوال روبه‌رو باتوجه به متن داده شده چیست؟ 
 سوال: شهربان و نقش آن در کنترل و نظارت برساخت و سازهای شهر چیست؟

متن: اشغال کلیه طبقات، تحت هر عنوان و به هر شکل، بیش از میزان تعیینشده در جداول چهارگانه استفاده از اراضی پهنههای اصلی و مستثنی شده در این بند، ممنوع است. (اصلاح طبق بند (۶) مصوبه مورخ ۰۵ / ۰۸ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۸ - ۳: در کلیه زیرپهنههای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنههای r ۲، s ۳، m ۲ و ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۸ - ۳: در کلیه زیرپهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای r ۲، s ۳، m ۲ و ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۸ - ۳: در کلیه زیرپهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای r ۲، s ۳، m ۲ و ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۸ - ۳: در کلیه زیرپهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای r ۲، s ۳، m ۲ و ۱۳۹۳ شورای عالی شهرسازی و معماری


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



بهترین پاسخ برای سوال روبه‌رو باتوجه به متن داده شده چیست؟ 
 سوال: هزینه مشاعات ساختمان بر عهده کیست؟ 

متن: تامین فضاهای خدماتی مورد نیاز شهر اقدام مینماید طرح فوق میبایست به تصویب کمیسیون ماده پنج شهر تهران برسند. (اصلاح طبق بند (۱) مصوبه مورخ ۲۸ / ۰۷ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۱۶ - ۲۲: سازمانهای اجرایی ذیربط موظف به همکاری با شهرداری تهران برای تهیه و تکمیل لایههای حرایم طبیعی و مصنوع در نقشههای طرح تفصیلی جدید شهر تهران میباشند. ۱۶ - ۲۲: سازمانهای اجرایی ذیربط موظف به همکاری با شهرداری تهیه و تکمیل لایههای حرایم طبیعی و مصنوع در نقشههای طرح تفصیلی جدید شهر تهران میباشند. ۱۶ - ۲۲: سازمانهای اجرایی ذیربط موظف به همکاری با شهرداری تهیه و تکمیل لایههای حرایم طبیعی و مصنوع در نقشههای طرح تفصیلی جدید شهر تهران میباشند. ۱۶ - ۲۲: سازمانهای اجرایی ذیربط موظف به همکاری با شهرداری تهیه و تکمیل لایههای حرایم طبیعی و مصنوع در نقشههای طرح تفصیلی جدید شهر تهران میباش


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


بهترین پاسخ برای سوال روبه‌رو باتوجه به متن داده شده چیست؟ 
 سوال: مدارک لازم جهت صدور فیش های درآمد و نوسازی

متن: تامین فضاهای خدماتی مورد نیاز شهر اقدام مینماید طرح فوق میبایست به تصویب کمیسیون ماده پنج شهر تهران برسند. (اصلاح طبق بند (۱) مصوبه مورخ ۲۸ / ۰۷ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۱۶ - ۲۲: سازمانهای اجرایی ذیربط موظف به همکاری با شهرداری تهران برای تهیه و تکمیل لایههای حرایم طبیعی و مصنوع در نقشههای طرح تفصیلی جدید شهر تهران میبایست به تصویب کمیسیون ماده پنج شهر تهران برسند. (اصلاح طبق بند (۱) مصوبه مورخ ۲۸ / ۰۷ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۱۶ - ۲۲: سازمانهای اجرایی ذیربط موظف به همکاری با شهرداری تهیه و تکمیل لایههای حرایم طبیعی و مصنوع در نقشههای طرح تفصیلی جدید شهر تهران میبایست به تصویب کمیسیون ماده پنج شهر تهران برسند. (اصلاح طبق بند (۱) مصوبه مورخ ۲۸ / ۰۷ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۱۶ - ۲۲: سازمانهای اجرایی ذیربط موظف به همکاری با ش


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



بهترین پاسخ برای سوال روبه‌رو باتوجه به متن داده شده چیست؟ 
 سوال: آیا احداث استخر و سونا و جکوزی و مشتقات آن در زیرزمین مجاز است؟

متن: مجاز است. ۲ - ۷: میزان سطح اشغال زیرزمین اول (منفی یک) ساختمانهای مسکونی حداکثر معادل سطح اشغال طبقه همکف به شرط عدم قطع درختان، مجاز است. میزان سطح اشغال در زیرزمینهای دوم (منفی دو) به پایین، در کلیه اراضی و املاک واقع در پهنه (r) حداکثر ۸۰ ٪ مساحت ملک پس از رعایت بر اصلاحی صرفا جهت احداث پارکینگ و مشاعات مربوطه، با بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر بر


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



بهترین پاسخ برای سوال روبه‌رو باتوجه به متن داده شده چیست؟ 
 سوال: دسترسی و نورگیری از فضای سبز عمومی چگونه است؟

متن: اشغال کلیه طبقات، تحت هر عنوان و به هر شکل، بیش از میزان تعیینشده در جداول چهارگانه استفاده از اراضی پهنههای اصلی و مستثنی شده در این بند، ممنوع است. (اصلاح طبق بند (۶) مصوبه مورخ ۰۵ / ۰۸ / ۱۳۹۳ شورای عالی شهرسازی و معماری ایران) ۸ - ۳: در کلیه زیرپهنههای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنههای r ۲، s ۳، m ۲ و g، با کلیه زیرپهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای استفاده از اراضی، به استثنای محدودههای تاریخی و پهنهای استفاده از اراضی، به استثنای محدودههای �

بهترین پاسخ برای سوال روبه‌رو باتوجه به متن داده شده چیست؟ 
 سوال: مدارک مورد نیاز برای پیش نویس پایان ک